In [10]:
import json, os
from proteinbert import load_pretrained_model
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
import numpy as np
import glob

seq_len = 3072
pretrained_model_generator, input_encoder = load_pretrained_model()
model = get_model_with_hidden_layers_as_outputs(pretrained_model_generator.create_model(seq_len))

seq_path = "C:\\Users\\efeka\\Documents\\GitHub\\549-term-project\\alignment\\all_sequences.json"

with open(seq_path, 'r') as f:
    sequences = json.load(f)


for sequence in sequences:
    ref = sequence.split('-')[0]
    msa = sequence.split('-')[1]
    sequence_name = sequence.split('-')[2]
    if ref == "RV912":
        #print(sequence)
        seq_t = sequences[sequence]

        encoded_x = input_encoder.encode_X([seq_t], seq_len)
        local_representations, _ = model.predict(encoded_x, batch_size = 1, verbose = 0)

        # SPECIAL TOKENS: 22: UNK, 23: BOS, 24: EOS, 25: PAD
        mask_to_exclude_special_tokens =  encoded_x[0][0] < 23 # exclude 23: BOS, 24: EOS, 25: PAD
        local_rep = local_representations[0][mask_to_exclude_special_tokens]
        
        # Save txt files of representations
        np.savetxt(f"output_representations/{ref}/{msa}/{sequence_name}.txt", local_rep[:,-154:-26])

In [5]:
local_rep.shape

(763, 128)